In [31]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd 

dataset = load_dataset("sms_spam")
dataset = dataset.rename_column("sms", "text")
dataset = dataset['train']

processed_text = dataset['text']
display(type(processed_text))
processed_text = [s.replace('\n', '') for s in processed_text]

dataset = dataset.remove_columns(["text"])
display(dataset)
dataset = dataset.add_column("text", processed_text)
display(dataset)
dataset = dataset.shuffle(seed=42).select([i for i in list(range(5500))])


train_ds = Dataset.from_dict(dataset[0:4500])
validation_ds = Dataset.from_dict(dataset[4500:5000])
test_ds = Dataset.from_dict(dataset[5000:5500])


display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Reusing dataset sms_spam (C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.35it/s]


list

Dataset({
    features: ['label'],
    num_rows: 5574
})

Dataset({
    features: ['label', 'text'],
    num_rows: 5574
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c\cache-1dabca71be75cbef.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

,label,text
0,1,sports fans - get the latest sports news str* ...
1,0,It's justbeen overa week since we broke up and...
2,0,Not directly behind... Abt 4 rows behind ü...
3,0,"Haha, my legs and neck are killing me and my a..."
4,0,Me too baby! I promise to treat you well! I be...
...,...,...
4495,1,Free entry in 2 a wkly comp to win FA Cup fina...
4496,1,I am hot n horny and willing I live local to y...
4497,0,Me too watching surya movie only. . .after 6 p...
4498,0,"Good evening Sir, hope you are having a nice d..."


In [32]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [33]:
#creating synonyms list using wordnet
def synonyms_list_creation(target_word):
    synonyms_list = []
    try:
        for syn in wordnet.synsets(target_word):
            for lm in syn.lemmas():
                synonyms_list.append(lm.name())
        #print (set(synonyms_list))
        synonyms_list = list(set(synonyms_list))
        #transform _ to space
        synonyms_list = [s.replace('_', ' ') for s in synonyms_list]


        synonyms_list
        synonyms_list = [s for s in synonyms_list if ' ' not in s]
    
    except Exception as e:
        pass
    
    return synonyms_list

In [34]:
#senticnet part
senticnet_df = pd.read_excel(r'C:\Users\DELL\Text_Augmentation\senticnet\senticnet.xlsx',header=0)

In [35]:
#transform _ to space

senticnet_df['CONCEPT'] = senticnet_df['CONCEPT'].str.replace('_', ' ')


senticnet_df.set_index('CONCEPT',inplace=True)

print(senticnet_df)

               INTROSPECTION  TEMPER  ATTITUDE  SENSITIVITY PRIMARY EMOTION  \
CONCEPT                                                                       
abandon               -0.165     0.0     0.000        0.000     #melancholy   
abase                  0.000     0.0    -0.890        0.000       #loathing   
abash                  0.000     0.0    -0.341       -0.442           #fear   
abashment              0.000     0.0    -0.329        0.000        #dislike   
abate                  0.000     0.0    -0.339       -0.551           #fear   
...                      ...     ...       ...          ...             ...   
zotob worm             0.000     0.0     0.000       -0.958         #terror   
zotob wrestle          0.000     0.0     0.000       -0.962         #terror   
zotob wriggle          0.000     0.0     0.000       -0.918         #terror   
zotob writhe           0.000     0.0     0.000       -0.979         #terror   
zz                     0.000     0.0    -0.182      

In [36]:
#creating similar words list using senticnet
def similar_polarity_words_list_creation(target_word):
    if target_word in senticnet_df.index:
        INTROSPECTION = senticnet_df.loc[target_word, 'INTROSPECTION']
        TEMPER = senticnet_df.loc[target_word, 'TEMPER']
        ATTITUDE = senticnet_df.loc[target_word, 'ATTITUDE']
        SENSITIVITY = senticnet_df.loc[target_word, 'SENSITIVITY']





        extracted_df = senticnet_df[(senticnet_df['INTROSPECTION'] == INTROSPECTION) & 
                                   (senticnet_df['TEMPER'] == TEMPER) & (senticnet_df['ATTITUDE'] == ATTITUDE) & 
                                    (senticnet_df['SENSITIVITY'] == SENSITIVITY)]


        similar_polarity_words_list = extracted_df.index.values.tolist()
        similar_polarity_words_list = [s for s in similar_polarity_words_list if ' ' not in s]
    
    else:
        similar_polarity_words_list = []
    
    return similar_polarity_words_list

In [37]:
import itertools
from textattack.transformations import WordSwap
class Cognate_WordSwap(WordSwap):
    
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        synonyms_list = synonyms_list_creation(word)
        cognate_list = []
        try:
            cognate_list.append(similar_polarity_words_list_creation(word))
        except Exception as e:
            pass
        
        cognate_list.append(synonyms_list)
        cognate_list = list(itertools.chain.from_iterable(cognate_list))
        cognate_list = list(set(cognate_list))

        return cognate_list


In [38]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore
from textattack.augmentation import Augmenter


transformation = Cognate_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en'),
                BERTScore(min_bert_score = 0.9, model_name='bert-base-uncased', num_layers=None, score_type='f1', compare_against_original=True)]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

In [39]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):

    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)

time_end = time.perf_counter()
tim = time_end - time_sta

In [40]:
print(tim/3600)




2.8612072224722227


In [41]:
display(results_list)

[['sports fans - contract the latest variation news str* 2 ur mobile 1 wk FREE PLUS a devoid TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we break up and already our brains are going to mush!"],
 ['Not immediately behind... Abt 4 rows behind ü...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to end the night with a suntan, mean I could swing by in like an hour?'],
 ['Me too baby! I promise to treat you well! I bet you will issue good care of me...'],
 ['Enjoy ur life. . dear night'],
 ['Sday only joined.so preparation we started today:)'],
 ['Prof: you have exit in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U slumber all morning?'],
 ['Have your lunch and come rapidly and open the door:)'],
 ['Cramps stopped. leaving back to sleep'],
 ["MAKE SURE ALEX jazz HIS BIRTHDAY IS OVER IN FIFTEEN MINUTES AS FAR AS YOU'RE CONCERNED"],

In [42]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we broke up and already our brains are going to mush!"],
 ['Not directly behind... Abt 4 rows behind ü...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to end the night with a burn, think I could swing by in like an hour?'],
 ['Me too baby! I promise to treat you well! I bet you will take good care of me...'],
 ['Enjoy ur life. . Good night'],
 ['Sday only joined.so training we started today:)'],
 ['Prof: you have passed in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U sleeping all morning?'],
 ['Have your lunch and come quickly and open the door:)'],
 ['Cramps stopped. Going back to sleep'],
 ["MAKE SURE ALEX KNOWS HIS BIRTHDAY IS OVER IN FIFTEEN MINUTES AS FAR AS YOU'RE CONCERNED"],
 ['"GRAN ONLYFO

In [43]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,sports fans - get the latest sports news str* ...,sports fans - contract the latest variation ne...
1,It's justbeen overa week since we broke up and...,It's justbeen overa week since we break up and...
2,Not directly behind... Abt 4 rows behind ü...,Not immediately behind... Abt 4 rows behind ü...
3,"Haha, my legs and neck are killing me and my a...","Haha, my legs and neck are killing me and my a..."
4,Me too baby! I promise to treat you well! I be...,Me too baby! I promise to treat you well! I be...
...,...,...
4495,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
4496,I am hot n horny and willing I live local to y...,I am hot n grittier and willing I live local t...
4497,Me too watching surya movie only. . .after 6 p...,Me too watch surya movie only. . .after 6 pm v...
4498,"Good evening Sir, hope you are having a nice d...","undecomposed evening Sir, hope you are having ..."


In [44]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,sports fans - get the latest sports news str* ...,sports fans - contract the latest variation ne...
1,It's justbeen overa week since we broke up and...,It's justbeen overa week since we break up and...
2,Not directly behind... Abt 4 rows behind ü...,Not immediately behind... Abt 4 rows behind ü...
3,"Haha, my legs and neck are killing me and my a...","Haha, my legs and neck are killing me and my a..."
4,Me too baby! I promise to treat you well! I be...,Me too baby! I promise to treat you well! I be...
...,...,...
4424,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
4425,I am hot n horny and willing I live local to y...,I am hot n grittier and willing I live local t...
4426,Me too watching surya movie only. . .after 6 p...,Me too watch surya movie only. . .after 6 pm v...
4427,"Good evening Sir, hope you are having a nice d...","undecomposed evening Sir, hope you are having ..."


In [45]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\augmented_datasets\Cognate with Pos Verb_Noun=False and Min Bert Score = 0.9\run3\dataset.csv')